In [ ]:
import pandas as pd
import numpy as np
import os
import re
import json
import string
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.autonotebook import tqdm
from functools import partial
from wordcloud import WordCloud, STOPWORDS
import nltk
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
nlp.max_length = 4000000
from nltk.probability import FreqDist

In [ ]:
df = pd.read_csv("../input/bigger-govt-dataset-list/data_set_26897.csv")
df.head()

In [ ]:
len(df.title.unique())

In [ ]:
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
train.head()

In [ ]:
train.isna().sum()

In [ ]:
train.dataset_label[0]

In [ ]:
print("le dataset contient : ",len(train), "élément")
[print(col + ":" + str(len(train[col].unique()))) for col in train.columns ]

In [ ]:
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
sample_sub.head()

In [ ]:
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
def json_to_text(filename, train_files_path=train_files_path, output='text'):  
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title')) #get car dictionnaire
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
def extraire_texte(filename, train_files_path=train_files_path):  
    json_path = os.path.join(train_files_path, (filename+'.json'))
    contents = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:contents.append(data.get('text'))

    all_contents = ' '.join(contents)
    return all_contents

In [ ]:
train['text'] = train['Id'].apply(extraire_texte)

In [ ]:
train.head()

In [ ]:
train.dataset_label[8000]

In [ ]:
import re
print(re.findall("Trends in International Mathematics and Science Study.*",train.text[8000]))

In [ ]:
train.text[10000]

In [ ]:
sample_sub['text'] = sample_sub['Id'].apply(partial(extraire_texte, train_files_path=test_files_path))

In [ ]:
sample_sub.head()

In [ ]:
def text_cleaning(text):
    
    text = ''.join([k for k in text if k not in string.punctuation]) # enlever les ponctuation
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip() # convertir en minuscule
    text = re.sub("/'+/g", ' ', text) # enlever les caracteres speciaux 
    text = re.sub("\s+"," ", text) # enlever les grandes espaces
    
    return text

In [ ]:
train['text'] = train['text'].apply(text_cleaning)

In [ ]:
sample_sub['text'] = sample_sub['text'].apply(text_cleaning)

In [ ]:
from nltk.tokenize import word_tokenize

words = word_tokenize(' '.join(list( train['cleaned_label'].values))) 
stopwords=set(STOPWORDS)
no_stopwords = [ i for i in words if i not in stopwords]

In [ ]:
mots_frequents = FreqDist(no_stopwords).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white', stopwords=STOPWORDS).generate(str(mots_frequents))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('les 100 mots les plus fréquents dans "cleaned_label"', fontsize=50)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#def prepare_text(text, nlp=nlp):
    '''
    Returns the text after stop-word removal and lemmatization.
    text - Sentence to be processed
    nlp - Spacy NLP model
    '''
 #   doc = nlp(text)
  #  lemma_list = [token.lemma_ for token in doc if not token.is_stop]
   # lemmatized_sentence = ' '.join(lemma_list)
    
    #return lemmatized_sentence

In [ ]:
label_1 = [x.lower() for x in train['dataset_label'].unique()]
label_2 = [x.lower() for x in train['dataset_title'].unique()]
label_3 = [x.lower() for x in train['cleaned_label'].unique()]
label_4 = [x.lower() for x in df.title.unique()]

labels_existants = set(label_1 + label_2 + label_3 + label_4)
id_list = []
lables_list = []
for index, row in tqdm(sample_sub.iterrows()):
    sample_text = row['text']
    row_id = row['Id']
    temp_df = train[train['text'] == text_cleaning(sample_text)]
    cleaned_labels = temp_df['cleaned_label'].to_list()
    for known_label in labels_existants:
        if known_label in sample_text.lower():
            cleaned_labels.append(text_cleaning(known_label))
    cleaned_labels = [text_cleaning(x) for x in cleaned_labels]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(row_id)

In [ ]:
resultats = pd.DataFrame()
resultats['Id'] = id_list
resultats['PredictionString'] = lables_list
resultats.head()

In [ ]:
resultats["PredictionString"][0]

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission